#### referenced by - https://github.com/kuleshov/audio-super-res

# Training ASR model

In [1]:
import os
os.sys.path.append(os.path.abspath('.'))
os.sys.path.append(os.path.dirname(os.path.abspath('.')))
import numpy as np
import matplotlib
from asr_model import ASRNet
from model import default_opt
from io_utils import upsample_wav
from io_utils import load_h5
import tensorflow as tf
#matplotlib.use('Agg')

In [2]:
args = {
    'train'      : 'prep_audioset.h5',
    'val'        : 'prep_audioset.val.h5',
    'alg'        : 'adam',
    'epochs'     : 100,
    'logname'    : 'default_log_name',
    'layers'     : 4,
    'lr'         : 1e-3,
    'batch_size' : 4
}
print(tf.__version__)

1.5.0


In [3]:
# get data
X_train, Y_train = load_h5(args['train'])
X_val, Y_val = load_h5(args['val'])

List of arrays in input file: KeysView(<HDF5 file "prep_audioset.h5" (mode r)>)
Shape of X: (48, 8192, 1)
Shape of Y: (48, 8192, 1)
List of arrays in input file: KeysView(<HDF5 file "prep_audioset.val.h5" (mode r)>)
Shape of X: (48, 8192, 1)
Shape of Y: (48, 8192, 1)


In [4]:
# determine super-resolution level
n_dim, n_chan = Y_train[0].shape
print('number of dimension:',n_dim)
print('number of channel:',n_chan)
r = Y_train[0].shape[1] / X_train[0].shape[1]
print('r:',r)
assert n_chan == 1 # if not number of channel is not 0 -> Error assert!

number of dimension: 8192
number of channel: 1
r: 1.0


In [5]:
# create model
def get_model(args, n_dim, r, from_ckpt=False, train=True):
    """Create a model based on arguments"""
    
    if train:
        opt_params = {
            'alg' : args['alg'], 
            'lr' : args['lr'], 
            'b1' : 0.9, 
            'b2' : 0.999,
            'batch_size': args['batch_size'], 
            'layers': args['layers']}
    else: 
        opt_params = default_opt

    # create model
    model = ASRNet(from_ckpt=from_ckpt, n_dim=n_dim, r=r,opt_params=opt_params, log_prefix=args['logname'])
    return model

model = get_model(args, n_dim, r, from_ckpt=False, train=True)

Layers: 4
D-Block:  (?, 8192, 128)
D-Block:  (?, 8192, 256)
D-Block:  (?, 8192, 512)
D-Block:  (?, 8192, 512)
512 9
Bottle-Neck:  (?, 8192, 512)
(-1, n/2, 2f)
(?, 8192, 512)

(-1, n, f)
(?, 8192, 1024)

(-1, n, 2f)
(?, 16384, 512)

(?, 8192, 512)


ValueError: `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [TensorShape([Dimension(None), Dimension(16384), Dimension(512)]), TensorShape([Dimension(None), Dimension(8192), Dimension(512)])]

In [ ]:
# train model
model.fit(X_train, Y_train, X_val, Y_val, n_epoch=args['epochs'])